### 1. Configuracion e importacion de datos

In [1]:
# !pip install ipywidgets

In [2]:
#```python
import pandas as pd
import numpy as np
import ipywidgets as widgets
from IPython.display import display, HTML
import re

In [3]:
# fichers de datos de resultados
df_mesures = pd.read_csv('data2/measures.csv')
df_researchers = pd.read_csv('data2/researchers.csv')
df_segmentation = pd.read_csv('data2/segmentation.csv')
df_ngrams = pd.read_csv('data2/ngrams_researchers.csv')

In [4]:
df_segmentation

,Name of the Measure,Brief description,Number of Items,Target population,Age Range,Target Environment,Disponibilidad idioma
0,Highly Sensitive Person Scale - Revised (HSP-R),The Highly Sensitive Person Scale – Revised (H...,18.0,Adulto,Adultos (18+),Vida Adulta General,English
1,Highly Sensitive Person Scale (HSP),The Highly Sensitive Person Scale (HSP) is a 2...,27.0,Adulto,Adultos (18+),Vida Adulta General,English
2,Highly Sensitive Person Scale – Brief Version ...,The Highly Sensitive Person Scale – Brief Vers...,12.0,Adulto,Adultos (18+),Vida Adulta General,English
3,Highly Sensitive Child Scale (HSC),The Highly Sensitive Child (HSC) scale is a 12...,12.0,Niño/Adolescente (Autoreporte),8-18 años,Hogar/Escuela (General Infantil),English
4,Highly Sensitive Child Scale - Parent Version ...,The Highly Sensitive Child Scale - Parent Vers...,12.0,Padres (Evaluación de Niño),No especificado,Hogar/Familia,English
5,Highly Sensitive Child Scale – Extended Versio...,The Highly Sensitive Child – Extended Version ...,21.0,Niño/Adolescente (Autoreporte),8-18 años,Hogar/Escuela (General Infantil),"English, Dutch"
6,Highly Sensitive Child in School (HSC – School...,The Highly Sensitive Child in School (HSC – Sc...,6.0,Profesor (Evaluación de Niño),No especificado,Entorno Escolar,English
7,The Highly Sensitive Child – Rating System (HS...,The Highly Sensitive Child – Rating System (HS...,NaN,Observador (Evaluación de Niño),Aprox. three años,Preescolar/Laboratorio,English


In [5]:
df_researchers.head()

,Name,Title,Institution,Expertise,Discipline of research,Location,Contact
0,Bianca P. Acevedo,(Ph.D.),NaN,I am a social neuroscientist specializing in h...,Social and Health Psychology,Dept of Psychology and the Neuroscience Resear...,bacevedo@ucsb.edu
1,Loredana Addimando,(Associate Professor),NaN,"Loredana Addimando is a psychologist, senior r...","Educational Psychology, Organizational Psychol...",University of Applied Sciences and Arts of Sou...,loredana.addimando@supsi.ch
2,Kirsty Allan,(Ph.D Student),NaN,I am currently undertaking PhD research in con...,Psychology,"University of Northampton, Carnforth, UK",kla@kirstylucindaallan.com
3,Arthur Aron,(Ph.D.),NaN,My work focuses on the continued development o...,Psychology,"University of New York, Stony Brook, New York,...",arthur.aron@stonybrook.edu
4,Elaine Aron,(Ph.D.),NaN,My work focuses on the continued development o...,Psychology,"University of New York, Stony Brook, New York,...",arthur.aron@stonybrook.edu


#### preprocesado

facilitamos la busqueda de expertos extrayendo las listas de palabras clave para el filtrado

In [6]:
df_researchers['Expertise_Keywords'] = df_researchers['Expertise'].str.split(r',\s*|;\s*|/', expand=False).apply(lambda x: [item.strip().upper() for item in x] if isinstance(x, list) else [])

In [7]:
df_researchers.head()

,Name,Title,Institution,Expertise,Discipline of research,Location,Contact,Expertise_Keywords
0,Bianca P. Acevedo,(Ph.D.),NaN,I am a social neuroscientist specializing in h...,Social and Health Psychology,Dept of Psychology and the Neuroscience Resear...,bacevedo@ucsb.edu,[I AM A SOCIAL NEUROSCIENTIST SPECIALIZING IN ...
1,Loredana Addimando,(Associate Professor),NaN,"Loredana Addimando is a psychologist, senior r...","Educational Psychology, Organizational Psychol...",University of Applied Sciences and Arts of Sou...,loredana.addimando@supsi.ch,"[LOREDANA ADDIMANDO IS A PSYCHOLOGIST, SENIOR ..."
2,Kirsty Allan,(Ph.D Student),NaN,I am currently undertaking PhD research in con...,Psychology,"University of Northampton, Carnforth, UK",kla@kirstylucindaallan.com,[I AM CURRENTLY UNDERTAKING PHD RESEARCH IN CO...
3,Arthur Aron,(Ph.D.),NaN,My work focuses on the continued development o...,Psychology,"University of New York, Stony Brook, New York,...",arthur.aron@stonybrook.edu,[MY WORK FOCUSES ON THE CONTINUED DEVELOPMENT ...
4,Elaine Aron,(Ph.D.),NaN,My work focuses on the continued development o...,Psychology,"University of New York, Stony Brook, New York,...",arthur.aron@stonybrook.edu,[MY WORK FOCUSES ON THE CONTINUED DEVELOPMENT ...


lista unica de todos los expertise identificados para el widget

In [8]:
expertise_list = set()

for keywords in df_researchers['Expertise_Keywords']:
    expertise_list.update(keywords)
unique_expertise_inventory = sorted([item for item in expertise_list if item and len(item) > 2])

# unique_expertise_inventory

lista unica de segmentos por edad identificables para el widget

In [9]:
unique_segmentation_list = sorted(df_segmentation['Target population'].unique().tolist())
unique_segmentation_list

['Adulto',
 'Niño/Adolescente (Autoreporte)',
 'Observador (Evaluación de Niño)',
 'Padres (Evaluación de Niño)',
 'Profesor (Evaluación de Niño)']

### FORMULARIO INTERACTIVO

In [17]:
# --- Definicion d6e Widgets

age_widget = widgets.Dropdown(
    options=['Hasta 3 años', 'Entre 3 y 8 años', 'Entre 8 - 18 años', 'Más de 18 años'],
    value='Más de 18 años',
    description='Rango de Edad:'
)

environment_widget = widgets.Dropdown(
    options=['Pre-escolar', 'Escolar', 'Hogar/Familia', 'Vida adulta en general'],
    value='Escolar',
    description='Entorno:'
)
submit_button = widgets.Button(description="Analizar y Recomendar")

# Contenedor para mostrar el formulario
form_container = widgets.VBox([
    age_widget,
    environment_widget
])

# --- Contenedor y Output ---
output_area = widgets.Output()

display(HTML("<h2> Medidas </h2>"))
display(form_container, output_area)

Output()

In [ ]:
def generate_recommendations(output):
    """ Función que se ejecuta cuando el formulario cambia y genera las recomendaciones."""
    with ouput:
        output.clear_output()

        selected_age = age_widget.value
        selected_environment = age_environment.value

        recommended_mesures = df_mesures.head()

        display(HTML("<h3> Medidas </h3>"))
        display(recommended_mesures)
        return

# --- Enlazar la función a los cambios de los widgets ---
widgets.interactive(generate_recommendations,
                   age=age_widget,
                   environment=environment_widget
                  )

# Mostrar el formulario
form = widgets.VBOS([
    widgets.HBOX([age_widget, environment_widget]),
    environment_widget,
])

display(form, output)
    

        